In [1]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
from matplotlib import font_manager, rcParams
import warnings, logging
import re

In [ ]:
ads_list = pd.read_csv("/Users/Jiwon/Documents/GitHub/final_project/Jiwon/광고도메인리스트.csv")
time_df = pd.read_csv("/Users/Jiwon/Documents/GitHub/final_project/Jiwon/수정_시간별적립보고서(최종).csv")
click = pd.read_csv("/Users/Jiwon/Documents/GitHub/final_project/Jiwon/유저테이블.csv")
cv_goal = pd.read_csv("/Users/Jiwon/Documents/GitHub/final_project/Jiwon/cv_goal.csv")

# ads

## ads_portfolio

In [ ]:
# epire 컬럼 만들기

# 1. 기준일 설정
today = pd.to_datetime('today').date()

# 2. expire 컬럼 초기화 (기본값 0)
ads_list['expire'] = 0

# 3. 실제 날짜 형태 필터링 및 만료 체크
def check_expire(date_str):
    try:
        # 9999나 0000이 포함된 경우는 무기한으로 처리 (expire = 0)
        if '9999' in str(date_str) or '0000' in str(date_str):
            return 0
        
        # 실제 날짜로 변환 시도
        end_date = pd.to_datetime(date_str).date()
        
        # 오늘보다 이전이면 만료 (1), 이후면 유효 (0)
        return 1 if end_date < today else 0
        
    except:
        # 변환 실패하면 무기한으로 처리
        return 0

# 적용
ads_list['expire'] = ads_list['ads_edate'].apply(check_expire)
ads_list.head()

In [ ]:
# ads_seg 테이블 만들기

click['click_date'] = pd.to_datetime(click['click_date'])

ads_seg = click.groupby('ads_idx').agg({
    'mda_idx': 'nunique',        # 매체사 수
    'dvc_idx': 'nunique',        # 유니크 사용자 수 (있다면)
    'click_key': 'count',
    'conversion': 'sum',
    'ads_category': 'first',
    'domain': 'first',
    'ads_os_type': 'first',
    'ads_order': 'first',
    'ctit': ['mean', 'median'],
    'ads_rejoin_type': 'first',
    'contract_price': 'first',
    'media_price': 'first',
    'click_date': ['min', 'max']
}).reset_index()

# 컬럼명 변경
ads_seg.columns = ['ads_idx', 'media_count', 'user_count', 'total_clicks', 'total_conversions', 'ads_category', 'domain', 'ads_os_type', 
                   'ads_order', 'ctit_mean', 'ctit_median', 'ads_rejoin_type', 'contract_price', 'media_price', 'first_click', 'last_click']

# (ads_name) 컬럼 붙이기
ads_seg = ads_seg.merge(ads_list[['ads_idx', 'ads_name', 'ads_sdate', 'expire']], on='ads_idx', how='left')

# 광고 기간 및 일평균 전환수 계산
ads_seg['days_active'] = (ads_seg['last_click'] - ads_seg['first_click']).dt.days + 1
ads_seg['daily_avg_conversions'] = ads_seg['total_conversions'] / ads_seg['days_active']


# 추후에 시간 datetime으로 바꿔서 active 여부랑 기간 넣기.

In [ ]:
# cvr 구하기
# 마진 구하기

ads_seg['cvr'] = (ads_seg['total_conversions'] / ads_seg['total_clicks']).round(1)
ads_seg['margin'] = (ads_seg['contract_price'] - ads_seg['media_price'])
ads_seg['roi'] = (ads_seg['margin'] / ads_seg['media_price']).round(1)
ads_seg['total_net_return'] = ads_seg['margin'] * ads_seg['total_conversions']


In [ ]:
# 광고 사이즈 구분

def standard_tier_classification(df):
    # daily_clicks 계산
    df['daily_clicks'] = df['total_clicks'] / df['days_active']
    
    # 점수 계산 (동일)
    def get_percentile_score(value, series):
        if value >= series.quantile(0.99): return 4
        elif value >= series.quantile(0.90): return 3  
        elif value >= series.quantile(0.70): return 2
        elif value >= series.quantile(0.40): return 1
        else: return 0
    
    df['media_score'] = df['media_count'].apply(lambda x: get_percentile_score(x, df['media_count']))
    df['conv_score'] = df['daily_avg_conversions'].apply(lambda x: get_percentile_score(x, df['daily_avg_conversions']))
    df['clicks_score'] = df['daily_clicks'].apply(lambda x: get_percentile_score(x, df['daily_clicks']))
    df['stability_score'] = df['days_active'].apply(lambda x: get_percentile_score(x, df['days_active']))
    df['cvr_score'] = df['cvr'].apply(lambda x: get_percentile_score(x, df['cvr']))
    
    df['total_score'] = (df['media_score'] + df['conv_score'] + df['clicks_score'] + 
                        df['stability_score'] + df['cvr_score'])
    
    # 표준 분류 (피라미드 형태)
    def get_standard_tier(score, score_series):
        if score >= score_series.quantile(0.995): return 'MEGA'    # 상위 5%
        elif score >= score_series.quantile(0.80): return 'LARGE'  # 상위 20%
        elif score >= score_series.quantile(0.30): return 'MEDIUM' # 상위 70% (가장 많음)
        else: return 'SMALL'                                      # 하위 30%
    
    df['ads_size'] = df['total_score'].apply(lambda x: get_standard_tier(x, df['total_score']))
    
    return df


ads_seg = standard_tier_classification(ads_seg)

print("백분위 기준 분류:")
print(ads_seg['ads_size'].value_counts())
print(f"\n분류 비율:")
print((ads_seg['ads_size'].value_counts() / len(ads_seg) * 100).round(1))
print(ads_seg.head())

### cv_goal

In [ ]:
# M, A 컬럼 붙여 넣기

# 먼저 M, A 컬럼 생성
cv_goal['M'] = (cv_goal['sch_type'] == 'M').astype(int)
cv_goal['A'] = (cv_goal['sch_type'] == 'A').astype(int)

# ads_idx 기준으로 그룹화하여 집계
cv_goal_grouped = cv_goal.groupby('ads_idx').agg({
    'mda_idx_arr': 'first',  # 첫 번째 값 사용 (M과 A가 다를 수 있으니 확인 필요)
    'sch_sdatetime': 'first',  # 시작일은 첫 번째 값
    'sch_edatetime': 'first',  # 종료일은 첫 번째 값
    'sch_adv_pay': 'first',
    'sch_ads_pay': 'first',
    'sch_mda_pay': 'max',
    'sch_clk_num': 'first',
    'sch_turn_num': 'sum',  # turn_num은 합계가 적절할 수 있음
    'M': 'max',  # M이 하나라도 있으면 1
    'A': 'max'   # A가 하나라도 있으면 1
}).reset_index()

print(f"Original rows: {len(cv_goal)}")
print(f"After grouping: {len(cv_goal_grouped)}")
print("\nM, A 조합 분포:")
print(cv_goal_grouped[['M', 'A']].value_counts())


In [ ]:
# ads_seg 랑 조인

# ads_idx를 기준으로 조인
merged_df = ads_seg.merge(
    cv_goal_grouped[['ads_idx', 'mda_idx_arr', 'M', 'A']], 
    on='ads_idx', 
    how='left'  # 또는 'inner', 'outer' 등 필요에 따라
)

# 여러 컬럼을 한 번에 처리
merged_df = merged_df.fillna({
    'mda_idx_arr': 'None',
    'M': 0,
    'A': 0
})

In [ ]:
# M==1 인 것들 중에서  광고가 우선 배정되는 m__arr 와 실제 들어가는 광고 똑같은 것 알아보기 (jaccard 사용)

m_ads = merged_df[merged_df['M']==1]

# 1) time_df에서 ads_idx별 mda_idx 리스트 만들기
time_mda = (
    time_df
      .dropna(subset=['ads_idx', 'mda_idx'])
      .groupby('ads_idx')['mda_idx']
      .unique()            # np.ndarray
      .map(list)           # list로 변환
      .rename('mda_idx_from_time')
      .reset_index()
)

# 2) m_ads와 조인
out = m_ads.merge(time_mda, on='ads_idx', how='left')

# ------------------ 여기부터 교체/추가 ------------------

def normalize_mda_list(x):
    """여러 형태(x)가 와도 [아이디, ...] 형태의 평평한 리스트로 표준화"""
    # 결측/None 처리
    if x is None or (isinstance(x, float) and pd.isna(x)) or (isinstance(x, str) and x.strip().lower() in {"none", "nan", ""}):
        return []
    # 문자열이면 안전 파싱 시도
    if isinstance(x, str):
        try:
            x = ast.literal_eval(x)
        except Exception:
            # '1,2,3' 같은 단순 콤마 문자열 대응
            return [i.strip() for i in x.split(',') if i.strip() != '']

    # 튜플은 리스트로
    if isinstance(x, tuple):
        x = list(x)

    # 리스트 계열은 내부 원소 평탄화
    if isinstance(x, list):
        flat = []
        for e in x:
            if e is None or (isinstance(e, float) and pd.isna(e)):
                continue
            if isinstance(e, tuple) or isinstance(e, list):
                flat.extend(list(e))       # [(562,563)] → [562,563], [[562,563]] → [562,563]
            else:
                flat.append(e)
        # 중복 제거(순서 보존)
        seen = set()
        dedup = []
        for v in flat:
            key = str(v)                   # 타입 혼합 대비
            if key not in seen:
                seen.add(key)
                dedup.append(v)
        return dedup

    # 그 외 스칼라
    return [x]

# 두 컬럼 모두 동일 규칙으로 표준화
out['mda_idx_arr_list']  = out['mda_idx_arr'].apply(normalize_mda_list)
out['mda_idx_from_time'] = out['mda_idx_from_time'].apply(normalize_mda_list)

# 4) 비교용: 집합(순서 무시)
to_set = lambda v: set(map(str, v))   # 타입 혼합 대비 문자열화
out['arr_set']  = out['mda_idx_arr_list'].apply(to_set)
out['time_set'] = out['mda_idx_from_time'].apply(to_set)

# 5) 비교 결과
out['is_equal_unordered'] = (out['arr_set'] == out['time_set'])
out['missing_in_arr'] = (out['time_set'] - out['arr_set']).apply(lambda s: sorted(s))
out['extra_in_arr']   = (out['arr_set'] - out['time_set']).apply(lambda s: sorted(s))

def jaccard(a, b):
    if not a and not b:
        return 1.0
    return len(a & b) / len(a | b) if (a | b) else 0.0

out['jaccard'] = [jaccard(a, b) for a, b in zip(out['arr_set'], out['time_set'])]

# 우선순위 1순위가 실제 등장하는지
out['top_priority_in_time'] = [
    (str(lst[0]) in tset) if lst else False
    for lst, tset in zip(out['mda_idx_arr_list'], out['time_set'])
]

# 6) 보기용 요약
compare_cols = [
    'ads_idx', 'mda_idx_arr_list', 'mda_idx_from_time',
    'is_equal_unordered', 'missing_in_arr', 'extra_in_arr',
    'jaccard', 'top_priority_in_time'
]
result_compare = out[compare_cols]



In [ ]:
# 지정매체광고
# mda_idx_from_time이 mda_idx_arr_list에 포함되거나 같은것만 뽑는거

mask = [
    all(m in row['mda_idx_arr_list'] for m in row['mda_idx_from_time'])
    for _, row in result_compare.iterrows()
]
restricted_mda = result_compare[mask]

restricted_mda

### ads_pool

In [ ]:
# 지정매체광고x
# 이거 반지정 아예 아닌거 나눠야하나?

# 제외할 광고 아이디 (restricted_mda에서)
exclude_ids = set(restricted_mda['ads_idx'].unique())

# ads_seg에서 이 아이디들 아닌 행만 보기
ads_seg_filter = merged_df[ ~merged_df['ads_idx'].isin(exclude_ids) ].copy()

# (선택) 인덱스 리셋
ads_seg_filter = ads_seg_filter.reset_index(drop=True)

In [ ]:
# expire 안된 애들 골라내기

ads_pool= ads_seg_filter[ads_seg_filter['expire'] == 0]

## media_portfolio

In [ ]:
# 1단계: 매체사-광고 관계 추출
media_ads_mapping = click[['mda_idx', 'ads_idx']].drop_duplicates()

# 2단계: 광고 분류 정보 조인
media_ads_mapping = media_ads_mapping.merge(
    ads_seg[['ads_idx', 'ads_size']], 
    on='ads_idx', 
    how='left'
)

# 3단계: 매체사별 광고 레벨 집계
media_portfolio = media_ads_mapping.groupby(['mda_idx', 'ads_size']).size().unstack(fill_value=0)

# 4단계: 총 광고 수 및 비율 계산
media_portfolio['total_ads'] = media_portfolio.sum(axis=1)

# 각 레벨별 비율 계산
for level in ['MEGA', 'LARGE', 'MEDIUM', 'SMALL']:
    if level in media_portfolio.columns:
        media_portfolio[f'{level}_ratio'] = (
            media_portfolio[level] / media_portfolio['total_ads'] * 100
        ).round(1)

# 5단계: 결과 확인
print("매체사별 포트폴리오 (상위 10개):")
print(media_portfolio.sort_values('total_ads', ascending=False).head(10))

# 6단계: 매체사 특성 분석
print("\n매체사 포트폴리오 요약:")
print(f"총 매체사 수: {len(media_portfolio)}")
print(f"평균 광고 보유 수: {media_portfolio['total_ads'].mean():.1f}")
print(f"광고 10개 이상 보유 매체사: {(media_portfolio['total_ads'] >= 10).sum()}개")

In [ ]:
# 1. 매체사 기본 정보 
media_basic = click.groupby('mda_idx').agg({
    'dvc_idx': 'nunique',        
    'click_key': 'count',
    'conversion': 'sum',
    'click_date': ['min', 'max']
}).reset_index()

media_basic.columns = ['mda_idx', 'user_count', 'total_clicks', 'total_conversions', 'first_click', 'last_click']

# 기간 및 일평균 계산
media_basic['days_active'] = (media_basic['last_click'] - media_basic['first_click']).dt.days + 1
media_basic['daily_avg_conversions'] = media_basic['total_conversions'] / media_basic['days_active']

# 2. 매체사-광고 관계
media_ads_mapping = click[['mda_idx', 'ads_idx']].drop_duplicates()
media_ads_mapping = media_ads_mapping.merge(
    ads_seg[['ads_idx', 'ads_size']], 
    on='ads_idx', 
    how='left'
)

# 3. 포트폴리오 구성
media_portfolio_composition = media_ads_mapping.groupby(['mda_idx', 'ads_size']).size().unstack(fill_value=0)
media_portfolio_composition['total_ads'] = media_portfolio_composition.sum(axis=1)

# 비율 계산
for level in ['MEGA', 'LARGE', 'MEDIUM', 'SMALL']:
    if level in media_portfolio_composition.columns:
        media_portfolio_composition[f'{level}_ratio'] = (
            media_portfolio_composition[level] / media_portfolio_composition['total_ads'] * 100
        ).round(1)

# 4. 최종 합치기
media_portfolio = media_basic.merge(
    media_portfolio_composition, 
    on='mda_idx', 
    how='left'
)

media_portfolio


In [ ]:
# 1. 매체사-광고 관계에 광고 속성 정보 추가
media_ads_with_attributes = media_ads_mapping.merge(
    ads_list[['ads_idx', 'ads_category', 'ads_os_type', 'domain']], 
    on='ads_idx', 
    how='left'
)

# 2. 각 속성별로 매체사별 분포 계산
def calculate_attribute_distribution(df, attribute_col):
    # 매체사별 해당 속성 분포
    attr_dist = df.groupby(['mda_idx', attribute_col]).size().unstack(fill_value=0)
    
    # 총 광고 수로 나누어 비율 계산
    attr_dist_pct = attr_dist.div(attr_dist.sum(axis=1), axis=0) * 100
    
    # 컬럼명에 prefix 추가
    attr_dist_pct.columns = [f'{attribute_col}_{col}_pct' for col in attr_dist_pct.columns]
    
    return attr_dist_pct

# 3. 각 속성별 비율 계산
category_dist = calculate_attribute_distribution(media_ads_with_attributes, 'ads_category')
os_dist = calculate_attribute_distribution(media_ads_with_attributes, 'ads_os_type')  
domain_dist = calculate_attribute_distribution(media_ads_with_attributes, 'domain')

# 4. 모든 분포 정보를 기존 테이블에 조인
media_portfolio = media_portfolio.merge(
    category_dist, on='mda_idx', how='left'
).merge(
    os_dist, on='mda_idx', how='left'
).merge(
    domain_dist, on='mda_idx', how='left'
)

# 5. 결과 확인
print(f"최종 테이블 크기: {media_portfolio.shape}")
print("추가된 컬럼들:")
new_cols = [col for col in media_portfolio.columns if '_pct' in col]
print(new_cols[:10])  # 처음 10개만 출력

# 샘플 확인
print("\n매체사별 속성 분포 (상위 5개 매체사):")
sample_cols = ['mda_idx', 'total_ads'] + new_cols[:6]  # 샘플로 몇 개만
print(media_portfolio[sample_cols].head())

In [ ]:
# 1. click 데이터에서 순수익 계산
click['profit_per_conversion'] = click['contract_price'] - click['media_price']

# 2. 각 행별 실제 수익 계산 (전환이 발생한 경우만)
click['actual_profit'] = click['profit_per_conversion'] * click['conversion']

# 3. 매체사별 총 예상 수익 집계
media_expected_profit = click.groupby('mda_idx')['actual_profit'].sum().reset_index()
media_expected_profit.columns = ['mda_idx', 'expected_total_profit']

# 4. media_portfolio에 조인
media_portfolio = media_portfolio.merge(
    media_expected_profit, 
    on='mda_idx', 
    how='left'
)

# NaN 처리 (수익이 없는 매체사는 0)
media_portfolio['expected_total_profit'] = (
    media_portfolio['expected_total_profit'].fillna(0)
)

# 결과 확인
print("매체사별 예상 수익 상위 10개:")
top_profit = media_portfolio.sort_values('expected_total_profit', ascending=False)
print(top_profit[['mda_idx', 'total_ads', 'total_conversions', 'expected_total_profit']].head(10))

print(f"\n전체 매체사 예상 수익 합계: {media_portfolio['expected_total_profit'].sum():,.0f}")

### media_group

In [ ]:

def classify_media_companies(df, auto_thresholds=False, reduce_misc=True):
    """
    매체사를 6가지 유형으로 분류 (기타 최소화 옵션 포함)
    """
    THRESHOLDS = {
        'total_ads_large': 115,        # 상위 10% (대량처리)
        'total_ads_medium': 28,        # 상위 25% (안정공급)
        'daily_conv_high': 500,        # 상위 10% (고성능)
        'daily_conv_medium': 164,      # 상위 25% (중성능)
        'days_active_stable': 25,      # 안정적 활동
        'days_active_new': 15,         # 신규 기준
        'conversion_rate_low': 0.001,  # 어뷰징 의심(0.1%)
        'specialization_threshold': 50, # 도메인 특화 (%)
        'mega_specialization': 70,     # MEGA 특화
        'large_specialization': 80,    # LARGE 특화
        'min_performance': 10,         # 최소 성과
        'inactive_cutoff': '2025-08-23' # 비활성 기준일(8/23 포함 이전)
    }
    ABUSE_MIN_CLICKS = 1000

    media_df = df.copy()

    # 파생 지표
    media_df['conversion_rate'] = media_df['total_conversions'] / media_df['total_clicks'].replace(0, np.nan)
    media_df['conversion_rate'] = media_df['conversion_rate'].fillna(0)
    media_df['last_click_dt'] = pd.to_datetime(media_df['last_click'], errors='coerce')
    cutoff_date = pd.to_datetime(THRESHOLDS['inactive_cutoff'])

    # (선택) 분위수 기반 자동 튜닝
    def _maybe_autotune_thresholds(df, TH, enabled):
        if not enabled: return TH
        q = df.quantile
        tuned = TH.copy()
        tuned.update({
            'total_ads_large': int(q(0.90)['total_ads']),
            'total_ads_medium': int(q(0.75)['total_ads']),
            'daily_conv_high': float(q(0.90)['daily_avg_conversions']),
            'daily_conv_medium': float(q(0.75)['daily_avg_conversions']),
            'days_active_stable': max(14, int(q(0.75)['days_active'])),
            'days_active_new': int(q(0.25)['days_active']),
            'conversion_rate_low': max(0.001, float(df['conversion_rate'].quantile(0.05))),
        })
        return tuned

    THRESHOLDS = _maybe_autotune_thresholds(media_df, THRESHOLDS, auto_thresholds)

    def check_domain_specialization(row):
        cols = [c for c in media_df.columns if c.startswith('domain_') and c.endswith('_pct')]
        for col in cols:
            v = row.get(col, np.nan)
            if pd.notna(v) and v >= THRESHOLDS['specialization_threshold']:
                name = col.replace('domain_', '').replace('_pct', '')
                return f"{name}특화"
        return None

    def check_size_specialization(row):
        if pd.notna(row.get('MEGA_ratio')) and row['MEGA_ratio'] >= THRESHOLDS['mega_specialization']:
            return "MEGA특화"
        if pd.notna(row.get('LARGE_ratio')) and row['LARGE_ratio'] >= THRESHOLDS['large_specialization']:
            return "LARGE특화"
        return None

    def classify_single_media(row):
        # 1) 계약종료형 (최우선)
        if (row['days_active'] <= 7) or (pd.notna(row['last_click_dt']) and row['last_click_dt'] < cutoff_date):
            return "계약종료형"

        # 2) 품질관리형 (어뷰징 의심)
        if (row['total_clicks'] >= ABUSE_MIN_CLICKS) and (row['conversion_rate'] < THRESHOLDS['conversion_rate_low']):
            return "품질관리형"

        # 3) 대량처리형
        if (row['total_ads'] >= THRESHOLDS['total_ads_large'] and
            row['daily_avg_conversions'] >= THRESHOLDS['daily_conv_high'] and
            row['days_active'] >= THRESHOLDS['days_active_stable']):
            return "대량처리형"

        # 4) 특화전문형
        domain_spec = check_domain_specialization(row)
        size_spec = check_size_specialization(row)
        if (domain_spec or size_spec) and (row['days_active'] >= THRESHOLDS['days_active_new']):
            return f"특화전문형_{domain_spec or size_spec}"

        # 5) 신규개발형
        if (row['days_active'] <= THRESHOLDS['days_active_new'] and
            row['daily_avg_conversions'] >= THRESHOLDS['min_performance']):
            return "신규개발형"

        # 6) 안정공급형
        if (row['total_ads'] >= THRESHOLDS['total_ads_medium'] and
            row['daily_avg_conversions'] >= THRESHOLDS['daily_conv_medium'] and
            row['days_active'] >= THRESHOLDS['days_active_stable']):
            return "안정공급형"

        # ---- Fallback: 기타 줄이기 ----
        if reduce_misc:
            # (a) 특화 신호만 있는 경우 → 특화전문형(후보)
            if (domain_spec or size_spec):
                return f"특화전문형_{domain_spec or size_spec}(후보)"
            # (b) 거의 안정 조건에 근접 → 안정공급형(후보)
            if (row['days_active'] >= max(THRESHOLDS['days_active_new'], THRESHOLDS['days_active_stable'] - 5)) and \
               ((row['daily_avg_conversions'] >= THRESHOLDS['min_performance']) or
                (row['total_ads'] >= int(THRESHOLDS['total_ads_medium'] * 0.6))):
                return "안정공급형(후보)"
            # (c) 활동일이 짧으면 → 신규개발형
            if row['days_active'] <= THRESHOLDS['days_active_new']:
                return "신규개발형"
            # (d) 남은 케이스는 관리 필요로 흡수
            return "관리 필요"

        return "기타"

    media_df['classification'] = media_df.apply(classify_single_media, axis=1)
    media_df['basic_classification'] = media_df['classification'].apply(lambda x: x.split('_')[0] if '_' in x else x)

    return media_df


In [ ]:
mda_pf_class = classify_media_companies(media_portfolio)

In [ ]:
# mda_pf_class에서 매체사 980번 빼기
# 너무 수익 이런거 혼자 극단치 

mda_pf_class = mda_pf_class[mda_pf_class['mda_idx'] != 980].reset_index(drop=True)

In [ ]:
# 한글, 오류 해결

def setup_korean_font(preferred=None, verbose=True):
    # 우선순위 높은 후보들(설치된 것 중 첫 번째를 씁니다)
    candidates = (preferred or []) + [
        'AppleGothic',            # macOS
        'Malgun Gothic',          # Windows
        'NanumGothic', 'NanumBarunGothic', 'NanumGothicCoding',
        'Noto Sans CJK KR', 'Noto Sans KR',
        'Arial Unicode MS'
    ]
    available = {f.name for f in font_manager.fontManager.ttflist}
    for name in candidates:
        if name in available:
            rcParams['font.family'] = name
            rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
            if verbose:
                print(f'✅ Using font: {name}')
            break
    else:
        print('⚠️ 한글 폰트가 발견되지 않았습니다. NanumGothic 또는 Noto Sans KR 설치 후 다시 실행하세요.')

def silence_matplotlib_warnings():
    # 글리프 경고/기타 matplotlib 경고 숨기기
    warnings.filterwarnings("ignore", message=r"Glyph .* missing from font", category=UserWarning)
    warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")
    logging.getLogger('matplotlib').setLevel(logging.ERROR)

setup_korean_font()
silence_matplotlib_warnings()

In [ ]:

DF = mda_pf_class.copy()  # classify_media_companies(mda_pf) 결과

# 분류 라벨 정렬(보고서용 가독성)
preferred_order = [
    '대량처리형', '안정공급형', '안정공급형(후보)',
    '특화전문형', '신규개발형',
    '관리 필요', '품질관리형', '계약종료형'
]
classes_in_df = [c for c in preferred_order if c in DF['basic_classification'].unique()]
DF['basic_classification'] = pd.Categorical(DF['basic_classification'], classes_in_df, ordered=True)

# 사용할 지표 목록
metrics = {
    'expected_total_profit': '기대 수익',
    'user_count': '유저 수',
    'total_ads': '광고 수',
    'daily_avg_conversions': '일평균 전환',
    'conversion_rate': '전환율',
    'MEGA_ratio': 'MEGA 비중(%)',
    'LARGE_ratio': 'LARGE 비중(%)',
    'days_active': '활동 일수'
}

# 요약 테이블(개수/평균/중앙값/합계)
summary = DF.groupby('basic_classification').agg(
    count=('mda_idx','count'),
    expected_total_profit_sum=('expected_total_profit','sum'),
    expected_total_profit_mean=('expected_total_profit','mean'),
    expected_total_profit_median=('expected_total_profit','median'),
    user_count_mean=('user_count','mean'),
    user_count_median=('user_count','median'),
    total_ads_mean=('total_ads','mean'),
    daily_avg_conversions_mean=('daily_avg_conversions','mean'),
    conversion_rate_mean=('conversion_rate','mean'),
    MEGA_ratio_mean=('MEGA_ratio','mean'),
    LARGE_ratio_mean=('LARGE_ratio','mean'),
    days_active_mean=('days_active','mean')
).round(2)
display(summary)  # 노트북 환경이면 표로 보임

# ---------- 헬퍼 함수 ----------
def _bar(series, title, ylabel, rotate=0, fmt_pct=False):
    plt.figure()
    ax = series.plot(kind='bar')
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.set_xlabel('분류')
    if rotate:
        plt.xticks(rotation=rotate)
    if fmt_pct:
        # y축 퍼센트 표시
        vals = ax.get_yticks()
        ax.set_yticklabels([f"{v:.0f}%" for v in vals])
    plt.tight_layout()
    plt.show()

def _box_by_class(df, col, title, ylabel, clip_q=0.99, log=False):
    dat = df[['basic_classification', col]].dropna().copy()
    # 극단값 클리핑(박스플롯 스케일 안정화)
    upper = dat[col].quantile(clip_q)
    dat[col] = np.clip(dat[col], dat[col].min(), upper)
    plt.figure()
    # 하나의 축 안에서 카테고리별 박스플롯
    groups = [g[col].values for _, g in dat.groupby('basic_classification')]
    labels = [str(c) for c in dat['basic_classification'].cat.categories]
    plt.boxplot(groups, labels=labels, showfliers=False)
    plt.title(title)
    plt.ylabel(ylabel)
    if log:
        plt.yscale('log')
    plt.xticks(rotation=20)
    plt.tight_layout()
    plt.show()

def _scatter(df, x, y, title, xlabel, ylabel, size_col=None, alpha=0.5, logx=False, logy=False):
    dat = df[[x, y, 'basic_classification'] + ([size_col] if size_col else [])].dropna().copy()
    plt.figure()
    if size_col:
        # 마커 크기 표준화
        s = dat[size_col].replace(0, np.nan)
        s = 50 * (s / s.quantile(0.9)).clip(0.2, 3)  # 적당한 범위로 스케일
        plt.scatter(dat[x], dat[y], s=s, alpha=alpha)
    else:
        plt.scatter(dat[x], dat[y], alpha=alpha)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if logx: plt.xscale('log')
    if logy: plt.yscale('log')
    plt.tight_layout()
    plt.show()

# ---------- 1) 분류 분포(개수/비율) ----------
counts = DF['basic_classification'].value_counts().reindex(classes_in_df).fillna(0)
_bar(counts, '분류별 매체사 개수', '개수', rotate=20)

pct = (counts / counts.sum() * 100).round(1)
_bar(pct, '분류별 매체사 비율', '비율(%)', rotate=20, fmt_pct=True)

# ---------- 2) 분류별 기대수익 ----------
# 합계(시장 기여도)
profit_sum = DF.groupby('basic_classification')['expected_total_profit'].sum().reindex(classes_in_df)
_bar(profit_sum, '분류별 기대수익 합계', '기대수익 합계', rotate=20)

# 평균/중앙값(단위 매체 기대수익)
profit_mean = DF.groupby('basic_classification')['expected_total_profit'].mean().reindex(classes_in_df)
_bar(profit_mean, '분류별 기대수익 평균', '평균 기대수익', rotate=20)

profit_median = DF.groupby('basic_classification')['expected_total_profit'].median().reindex(classes_in_df)
_bar(profit_median, '분류별 기대수익 중앙값', '중앙 기대수익', rotate=20)

# 분포(대수 스케일 권장)
_box_by_class(DF, 'expected_total_profit', '분류별 기대수익 분포(박스플롯)', '기대수익', clip_q=0.99, log=True)

# ---------- 3) 유저 수 ----------
user_mean = DF.groupby('basic_classification')['user_count'].mean().reindex(classes_in_df)
_bar(user_mean, '분류별 유저 수 평균', '평균 유저 수', rotate=20)

_box_by_class(DF, 'user_count', '분류별 유저 수 분포(박스플롯)', '유저 수', clip_q=0.99, log=True)

# ---------- 4) 운영 볼륨(광고 수, 일평균 전환) ----------
ads_mean = DF.groupby('basic_classification')['total_ads'].mean().reindex(classes_in_df)
_bar(ads_mean, '분류별 광고 수 평균', '평균 광고 수', rotate=20)

_box_by_class(DF, 'daily_avg_conversions', '분류별 일평균 전환 분포(박스플롯)', '일평균 전환', clip_q=0.99, log=True)

# ---------- 5) 전환율 ----------
cr_mean = DF.groupby('basic_classification')['conversion_rate'].mean().reindex(classes_in_df)
_bar(cr_mean, '분류별 전환율 평균', '평균 전환율', rotate=20)

_box_by_class(DF, 'conversion_rate', '분류별 전환율 분포(박스플롯)', '전환율', clip_q=0.99, log=False)

# ---------- 6) 포맷 특성(MEGA/LARGE 비중) ----------
mega_mean = DF.groupby('basic_classification')['MEGA_ratio'].mean().reindex(classes_in_df)
_bar(mega_mean, '분류별 MEGA 비중 평균(%)', 'MEGA 비중(%)', rotate=20)

large_mean = DF.groupby('basic_classification')['LARGE_ratio'].mean().reindex(classes_in_df)
_bar(large_mean, '분류별 LARGE 비중 평균(%)', 'LARGE 비중(%)', rotate=20)

# ---------- 7) 활동성 ----------
days_mean = DF.groupby('basic_classification')['days_active'].mean().reindex(classes_in_df)
_bar(days_mean, '분류별 활동 일수 평균', '평균 활동 일수', rotate=20)

_box_by_class(DF, 'days_active', '분류별 활동 일수 분포(박스플롯)', '활동 일수', clip_q=0.99, log=False)

# ---------- 8) 관계형 시각화(산점도) ----------
# 기대수익 vs 유저 수 (마커 크기: 광고 수)
_scatter(DF, 'user_count', 'expected_total_profit',
         '기대수익 vs 유저 수 (마커 크기=광고 수)',
         '유저 수', '기대수익', size_col='total_ads', logx=True, logy=True)

# 기대수익 vs 일평균 전환
_scatter(DF, 'daily_avg_conversions', 'expected_total_profit',
         '기대수익 vs 일평균 전환', '일평균 전환', '기대수익', logx=True, logy=True)

# 전환율 vs 기대수익
_scatter(DF, 'conversion_rate', 'expected_total_profit',
         '기대수익 vs 전환율', '전환율', '기대수익', logx=False, logy=True)


# 매체사별 광고 성과

In [ ]:
def analyze_ads_performance(ads_idx, click_data, media_portfolio=None):
    """
    특정 광고의 매체별 성과를 분석하는 함수
    """
    
    # 1. 해당 광고의 데이터가 있는지 확인
    ads_data = click_data[click_data['ads_idx'] == ads_idx]
    if len(ads_data) == 0:
        print(f"광고 {ads_idx}에 대한 데이터가 없습니다.")
        return pd.DataFrame()
    
    # 2. 기본 성과 데이터 추출
    ads_performance = ads_data.groupby(['ads_idx', 'mda_idx']).agg({
        'click_key': 'count',
        'conversion': 'sum',
        'contract_price': 'first',
        'media_price': 'first',
        'domain': 'first',
        'ads_category': 'first'
    }).reset_index()
    
    # 컬럼명 변경
    ads_performance.columns = ['ads_idx', 'mda_idx', 'total_clicks', 'total_conversions', 
                              'contract_price', 'media_price', 'domain', 'ads_category']
    
    # 전환율 및 수익 계산
    ads_performance['cvr'] = (
        ads_performance['total_conversions'] / ads_performance['total_clicks']
    ).round(4)
    
    ads_performance['profit_per_conversion'] = (
        ads_performance['contract_price'] - ads_performance['media_price']
    )
    ads_performance['total_profit'] = (
        ads_performance['total_conversions'] * ads_performance['profit_per_conversion']
    )
    
    # 3. 날짜 범위 및 활동일 계산
    click_data_copy = click_data.copy()
    if not pd.api.types.is_datetime64_any_dtype(click_data_copy['click_date']):
        click_data_copy['click_date'] = pd.to_datetime(click_data_copy['click_date'])
    
    ads_activity = (
        click_data_copy.loc[click_data_copy['ads_idx'] == ads_idx]
                      .groupby('mda_idx')['click_date']
                      .agg(first_click='min', last_click='max')
                      .reset_index()
    )
    
    ads_activity['days_active_calc'] = (
        (ads_activity['last_click'] - ads_activity['first_click']).dt.days + 1
    )
    
    # 4. 데이터 병합
    merged = ads_performance.merge(
        ads_activity[['mda_idx', 'first_click', 'last_click', 'days_active_calc']],
        on='mda_idx', how='left'
    )
    
    # 5. 일평균 지표 계산
    merged['daily_clicks'] = merged['total_clicks'] / merged['days_active_calc']
    merged['daily_conversions'] = merged['total_conversions'] / merged['days_active_calc']
    merged['daily_profit'] = merged['total_profit'] / merged['days_active_calc']
    
    # 6. 배분 그룹 분류 (데이터가 충분한 경우에만)
    if len(merged) > 1:  # 최소 2개 이상의 매체가 있어야 중앙값 계산이 의미있음
        profit_median = merged['daily_profit'].median()
        conv_median = merged['daily_conversions'].median()
        
        merged['배분그룹'] = np.where(
            (merged['daily_profit'] >= profit_median) & (merged['daily_conversions'] >= conv_median),
            '잘 배분',
            '잘못 배분'
        )
        # 결과 정렬
        result = merged.sort_values(['배분그룹', 'daily_profit'], ascending=[True, False]).reset_index(drop=True)
    else:
        merged['배분그룹'] = '분류불가'
        result = merged.reset_index(drop=True)
    
    return result

# 매체사 유사도

In [ ]:
# 매체사 포트폴리오 각 매체사별 카테고리나 도메인 전환 비중 


# 컬럼 이름에서 공백/특수문자 → '_' 로 바꾸는 헬퍼
def _slug(s):
    return re.sub(r'[^0-9A-Za-z가-힣]+', '_', str(s)).strip('_')

def add_cat_domain_to_mda_pf(
    mda_pf: pd.DataFrame,
    clicks_df: pd.DataFrame,
    conv_col: str = "conversion",
    cat_col: str = "ads_category",
    dom_col: str = "domain",
    add_within_cat: bool = False,      # mda×카테고리 내부 도메인 구성비 추가 여부
    add_within_dom: bool = False       # mda×도메인 내부 카테고리 구성비 추가 여부
):
    """
    반환: (enriched_mda_pf, new_columns)
    - conv_cat{카테고리}_{도메인} : 해당 mda의 (카테고리×도메인) 전환수
    - share_cat{카테고리}_{도메인}: 해당 mda 전체 전환 대비 구성비(0~1)
    - (옵션) shareWithinCat_*, shareWithinDomain_* 도 함께 추가 가능
    """
    df = clicks_df.copy()

    # 전환수 정리 (0/1이 아니면 그대로 합산, 0/1이면 1 합산)
    df[conv_col] = pd.to_numeric(df[conv_col], errors="coerce").fillna(0)
    if df[conv_col].max() <= 1:
        df["conv"] = (df[conv_col] > 0).astype(int)
    else:
        df["conv"] = df[conv_col]

    # 전환 있는 행만
    conv = df[df["conv"] > 0].copy()
    if conv.empty:
        enriched = mda_pf.copy()
        return enriched, []

    # mda × category × domain 전환수 집계
    g = (conv.groupby(["mda_idx", cat_col, dom_col], as_index=False)["conv"]
              .sum())

    # mda 전체 전환 합 → mda 대비 구성비
    total_mda = (g.groupby("mda_idx", as_index=False)["conv"]
                   .sum()
                   .rename(columns={"conv":"total_mda"}))
    g = g.merge(total_mda, on="mda_idx", how="left")
    g["share_mda"] = g["conv"] / g["total_mda"].replace(0, np.nan)
    g["share_mda"] = g["share_mda"].fillna(0.0)

    # ---- 피벗: 전환수 / mda-구성비
    piv_cnt = (g.pivot(index="mda_idx",
                       columns=[cat_col, dom_col],
                       values="conv")
                 .fillna(0))
    piv_shr = (g.pivot(index="mda_idx",
                       columns=[cat_col, dom_col],
                       values="share_mda")
                 .fillna(0.0))

    # 컬럼 평탄화
    piv_cnt.columns = [f"conv_cat{c}_{_slug(d)}" for c, d in piv_cnt.columns]
    piv_shr.columns = [f"share_cat{c}_{_slug(d)}" for c, d in piv_shr.columns]

    out = (mda_pf.merge(piv_cnt, on="mda_idx", how="left")
                 .merge(piv_shr, on="mda_idx", how="left"))

    new_cols = list(piv_cnt.columns) + list(piv_shr.columns)
    out[new_cols] = out[new_cols].fillna(0)

    # ---- (옵션) mda×카테고리 내부 도메인 구성비
    if add_within_cat:
        tot_cat = (g.groupby(["mda_idx", cat_col], as_index=False)["conv"]
                     .sum()
                     .rename(columns={"conv":"_tot_cat"}))
        g2 = g.merge(tot_cat, on=["mda_idx", cat_col], how="left")
        g2["share_within_cat"] = g2["conv"] / g2["_tot_cat"].replace(0, np.nan)
        piv_wc = (g2.pivot(index="mda_idx",
                           columns=[cat_col, dom_col],
                           values="share_within_cat")
                    .fillna(0.0))
        piv_wc.columns = [f"shareWithinCat_cat{c}_{_slug(d)}" for c, d in piv_wc.columns]
        out = out.merge(piv_wc, on="mda_idx", how="left")
        out[piv_wc.columns] = out[piv_wc.columns].fillna(0.0)
        new_cols += list(piv_wc.columns)

    # ---- (옵션) mda×도메인 내부 카테고리 구성비
    if add_within_dom:
        tot_dom = (g.groupby(["mda_idx", dom_col], as_index=False)["conv"]
                     .sum()
                     .rename(columns={"conv":"_tot_dom"}))
        g3 = g.merge(tot_dom, on=["mda_idx", dom_col], how="left")
        g3["share_within_domain"] = g3["conv"] / g3["_tot_dom"].replace(0, np.nan)
        piv_wd = (g3.pivot(index="mda_idx",
                           columns=[cat_col, dom_col],
                           values="share_within_domain")
                    .fillna(0.0))
        piv_wd.columns = [f"shareWithinDomain_cat{c}_{_slug(d)}" for c, d in piv_wd.columns]
        out = out.merge(piv_wd, on="mda_idx", how="left")
        out[piv_wd.columns] = out[piv_wd.columns].fillna(0.0)
        new_cols += list(piv_wd.columns)

    return out, new_cols


In [ ]:
# clicks_df: 원본 클릭/전환 테이블 (mda_idx, ads_category, domain, conversion 포함)
# mda_pf: 매체 프로필 테이블 (mda_idx 기준)

mda_pf_enriched, added_cols = add_cat_domain_to_mda_pf(
    mda_pf, click,
    add_within_cat=False,     # 필요하면 True
    add_within_dom=False      # 필요하면 True
)

print(f"추가된 컬럼 수: {len(added_cols)}")
# mda_pf_enriched.head()


In [ ]:
# 유사도

# --- 유틸 ---
def _slug(s): 
    return re.sub(r'[^0-9A-Za-z가-힣]+', '_', str(s)).strip('_')

def _cosine(a, B):
    a = a.reshape(1, -1)
    num = (B * a).sum(axis=1)
    den = (np.sqrt((B**2).sum(axis=1)) * np.sqrt((a**2).sum()))
    den = np.where(den == 0, 1e-12, den)
    return (num / den).ravel()

# --- 광고 데이터로 (카테고리×도메인) 가중치 만들기 ---
def make_ad_pair_weights_from_ad_df(ad_df, cat_col='ads_category', dom_col='domain',
                                    conv_col='total_conversions', power=1.0, min_frac=0.0):
    """
    ad_df에서 (카테고리×도메인)별 전환 비중 → share_cat{c}_{domain} 컬럼의 가중치 dict 반환
    - power: 비중에 지수 가중 (1.0=그대로, 0.5=루트, 2.0=제곱)
    - min_frac: 너무 작은 비중 컷(0~1)
    """
    t = ad_df.copy()
    t[conv_col] = pd.to_numeric(t[conv_col], errors='coerce').fillna(0.0)
    g = (t.groupby([cat_col, dom_col])[conv_col].sum()
           .rename('conv').reset_index())
    tot = g['conv'].sum()
    if tot <= 0:
        return {}
    g['frac'] = g['conv'] / tot
    if min_frac > 0:
        g = g[g['frac'] >= min_frac].copy()
    g['w'] = (g['frac'] ** power)
    # 정규화(합=1)
    s = g['w'].sum()
    if s > 0:
        g['w'] = g['w'] / s
    # share_cat{c}_{slug(domain)} 키로 변환
    weights = { f"share_cat{int(c)}_{_slug(d)}": float(w) for c,d,w in g[[cat_col, dom_col, 'w']].itertuples(index=False) }
    return weights

# --- 피처 행렬 만들기 (z-score + 컬럼별 가중적용) ---
def build_feature_matrix(mda_pf, feature_cols, col_weights=None, zscore=True):
    X = mda_pf.set_index('mda_idx')[feature_cols].astype(float).fillna(0.0)
    if col_weights:
        w = np.array([col_weights.get(c, 1.0) for c in feature_cols], dtype=float)
        X = X * w  # 가중치 적용 (열 스케일)
    if zscore:
        X = (X - X.mean()) / (X.std() + 1e-9)
    return X

# --- 추천 메인 ---
def recommend_with_weighted_similarity(
    ad_df,               # 특정 광고의 매체 성과 테이블(ads_XXXX_pf)
    mda_pf,              # (enriched) 전체 매체 프로필 (share_cat* 들어있는 테이블)
    top_anchor_by='total_conversions',
    n_anchor=3,
    topN=20,
    weight_power=1.0,    # 광고 전환 분포 가중치 지수
    min_pair_frac=0.0,   # 광고 전환 분포에서 너무 작은 비중 컷
    top_weight_feats=None,  # 상위 몇 개 가중치 feature만 사용할지 (None이면 전체)
    exclude_classes=('계약종료형','품질관리형'),  # 운영상 제외할 타입
    min_days_active=7,
    blend_pred_table=None,   # pred_table(ads_idx,mda_idx,pred_turn)이 있으면 넣기
    blend_ad_id=None,        # blend할 광고 ID
    blend_alpha=0.7          # 최종점수 = alpha*similarity + (1-alpha)*pred_norm
):
    # 1) share_cat* 피처만 사용 (전환 '비중' 기반 유사도에 초점)
    share_cols = [c for c in mda_pf.columns if c.startswith('share_cat')]
    if not share_cols:
        raise ValueError("mda_pf에 share_cat* 컬럼이 없습니다. 먼저 enrichment를 수행하세요.")

    # 2) 광고 전환 분포 기반 컬럼 가중치(없으면 균등 가중)
    col_w = make_ad_pair_weights_from_ad_df(ad_df, power=weight_power, min_frac=min_pair_frac)
    if top_weight_feats:
        # 광고에서 의미 있는 상위 조합만 남기고 나머지는 0으로 눌러서 노이즈 감소
        top_keys = set(pd.Series(col_w).sort_values(ascending=False).head(top_weight_feats).index)
        col_w = {k: (v if k in top_keys else 0.0) for k,v in col_w.items()}

    # 3) 피처 행렬(z-score + 가중치)
    X = build_feature_matrix(mda_pf, share_cols, col_weights=col_w, zscore=True)

    # 4) 앵커(이 광고에서 상위 성과 매체)
    used = set(ad_df['mda_idx'].astype(int))
    anchors = (ad_df.sort_values(top_anchor_by, ascending=False)
                   .drop_duplicates('mda_idx')
                   .head(n_anchor)['mda_idx']
                   .astype(int).tolist())
    anchors = [m for m in anchors if m in X.index]
    if not anchors:
        raise ValueError("anchor가 없습니다. ad_df에 상위 매체가 있는지 확인하세요.")

    centroid = X.loc[anchors].mean(axis=0).values

    # 5) 후보: 미사용 매체 + 운영 필터
    cand = mda_pf[~mda_pf['mda_idx'].isin(used)].copy()
    if 'basic_classification' in cand.columns and exclude_classes:
        cand = cand[~cand['basic_classification'].isin(exclude_classes)]
    if 'days_active' in cand.columns:
        cand = cand[cand['days_active'] >= min_days_active]
    if cand.empty:
        return pd.DataFrame(columns=['mda_idx','similarity']), anchors, share_cols, col_w

    # 6) 유사도 계산 (가중 코사인)
    B = X.loc[cand['mda_idx']].values
    sims = cosine_vec(centroid, B)
    cand['similarity'] = sims

    # # 7) (선택) 예측 전환수와 블렌딩
    # if blend_pred_table is not None and blend_ad_id is not None:
    #     pt = blend_pred_table[blend_pred_table['ads_idx']==blend_ad_id][['mda_idx','pred_turn']].copy()
    #     cand = cand.merge(pt, on='mda_idx', how='left')
    #     cand['pred_turn'] = cand['pred_turn'].fillna(0.0)
    #     # 간단 정규화
    #     maxv = cand['pred_turn'].max()
    #     cand['pred_norm'] = cand['pred_turn'] / (maxv + 1e-9)
    #     cand['final_score'] = blend_alpha*cand['similarity'] + (1.0-blend_alpha)*cand['pred_norm']
    #     sort_key = 'final_score'
    # else:
    #     sort_key = 'similarity'

    # 8) 보기 좋게 컬럼 추리기 + 정렬
    keep = [c for c in ['mda_idx','similarity','final_score','pred_turn','basic_classification',
                        'days_active','conversion_rate','expected_total_profit','total_ads'] if c in cand.columns]
    out = cand[keep].sort_values(sort_key, ascending=False).head(topN).reset_index(drop=True)
    return out, anchors, share_cols, col_w


In [ ]:
# ad_df = ads_73878_pf     # 이 광고의 매체별 성과 테이블
# mda_pf = mda_pf_enriched # share_cat* 컬럼 붙인 테이블

topN = 20
recs, anchors, used_feats, weight_map = recommend_with_weighted_similarity(
    ad_df=ads_73878_pf,
    mda_pf=mda_pf_enriched,                    # 위에서 이미 share_cat*가 들어간 버전
    top_anchor_by='total_conversions',
    n_anchor=3,
    topN=topN,
    weight_power=1.0,                 # 광고 전환 분포를 그대로 사용(필요 시 1.5~2.0으로 강화)
    min_pair_frac=0.0,                # 아주 작은 비중도 반영(잡음 줄이려면 0.01 같이 줘도 됨)
    top_weight_feats=40,              # 광고에서 중요한 조합 상위 40개만 유의하게 사용 (노이즈 컷)
    exclude_classes=('계약종료형','품질관리형'),  # 리스크 제외
    min_days_active=7,
    # # 있으면 켜기: 예측 전환수와 블렌딩
    # blend_pred_table=(pred_table if 'pred_table' in globals() else None),
    # blend_ad_id=73878 if 'pred_table' in globals() else None,
    # blend_alpha=0.7
)

print("anchors:", anchors)       # 이 광고에서 기준으로 삼은 상위 매체
recs.head(topN)                  # 최종 추천
